In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd
import warnings
from sklearn.neighbors import BallTree
import lightgbm as lgb
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)
input_path = '../input/'
gdf_land = gpd.read_file(os.path.join(input_path, 'L02-25.geojson'))
df_land = gdf_land.copy()
df_land['price'] = df_land['L02_006']
df = pd.read_csv(os.path.join(input_path, 'train.csv'))
test = pd.read_csv(os.path.join(input_path, 'test.csv'))
gdf_land.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 21431 entries, 0 to 21430
Columns: 140 entries, L02_001 to geometry
dtypes: float64(4), geometry(1), int32(69), object(66)
memory usage: 17.2+ MB


In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

def preprocess_data(input_df, df_land, is_train=True):
    """
    駅情報は一旦無視！
    「巨大なゴミデータ」の削除と、「地価・築年数」だけに集中したシンプル版
    """
    df = input_df.copy()
    
    # -----------------------------------------------------
    # 0. 【Trainのみ】ハズレ値などの削除
    # -----------------------------------------------------
    if is_train:
        # 1. 面積フィルタ
        # 面積500㎡以上はカット
        df = df[df['unit_area'] < 500] 
        
        # 2. 単価がおかしい土地を削除
        # 単価(円/㎡)を計算
        temp_unit_price = df['money_room'] / df['unit_area']
        
        # 「単価が1万円/㎡ 以下」は、原野や山林なので捨てる
        df = df[temp_unit_price > 10000] 
        
        # 3. 「高すぎる単価」もノイズになるのでキャップを設ける
        df = df[temp_unit_price < 10000000]

    # -----------------------------------------------------
    # 1. 駅情報 (今後実装予定)
    # -----------------------------------------------------
    # ※カラム名が画像通り (bus_time1, walk_distance1) 前提で書きます
    # もし違ったら df.columns で確認して書き換えてください！
    
    # 必要な列があるか確認
    if 'walk_distance1' in df.columns:
        
        def calc_access_time(row, suffix):
            """
            suffix: '1' または '2' を指定
            """
            # 1. データの取得 (欠損値は0にしておく)
            w_dist = row.get(f'walk_distance{suffix}', np.nan)
            b_time = row.get(f'bus_time{suffix}', np.nan)
            
            # データが全くない場合は「不明(無限大)」
            if pd.isna(w_dist) and pd.isna(b_time):
                return 999.0 # あとでminを取るときに負けるように大きくする

            # NaNを0扱いで計算用に確保
            w_dist_val = 0 if pd.isna(w_dist) else w_dist
            b_time_val = 0 if pd.isna(b_time) else b_time
            
            # 2. 徒歩時間を計算 (80m = 1分
            walk_min = w_dist_val / 80.0
            
            # 3. ロジック分岐
            if b_time_val > 0:
                # 【パターンA: バス利用】
                # 時間 = バス乗車時間 + バス停からの徒歩 + 待ち時間ペナルティ(10分)
                return b_time_val + walk_min + 10.0
            else:
                # 【パターンB: 徒歩のみ】
                # 時間 = 駅からの徒歩
                return walk_min

        # ルート1とルート2、それぞれの所要時間を計算
        df['access_time_1'] = df.apply(lambda x: calc_access_time(x, '1'), axis=1)
        df['access_time_2'] = df.apply(lambda x: calc_access_time(x, '2'), axis=1)
        
        # 「1」と「2」のうち、時間が短い方（＝近い方）を採用！
        # どっちも999(不明)なら、最終的に120分で埋める
        df['station_minutes'] = df[['access_time_1', 'access_time_2']].min(axis=1)
        
        # 最終的な欠損値埋め (田舎対策)
        # 999のまま残ったやつは「駅徒歩圏外」として120分にする
        df['station_minutes'] = df['station_minutes'].replace(999.0, 120.0)
        
        # 念のため station_minutes 自体がNaNのやつも埋める
        df['station_minutes'] = df['station_minutes'].fillna(120.0)

        # 不要になった一時的な列を削除（メモリ節約）
        df.drop(['access_time_1', 'access_time_2'], axis=1, inplace=True)
        
        # カテゴリ用 (沿線名などは _1 の方を正として採用しておく)
        if 'rosen_name1' in df.columns:
            df['line_name'] = df['rosen_name1']
            df['station_name'] = df['eki_name1']
        else:
            df['line_name'] = 'unknown'
            df['station_name'] = 'unknown'

    else:
        # そもそも列がない場合
        df['station_minutes'] = 120.0
        df['line_name'] = 'unknown'
        df['station_name'] = 'unknown'

    # -----------------------------------------------------
    # 2. 築年数の計算 
    # -----------------------------------------------------
    df['year_built'] = pd.to_numeric(df['year_built'], errors='coerce')
    df['year_built'] = df['year_built'].fillna(df['year_built'].median())
    df['temp_time'] = pd.to_datetime(df['year_built'].astype(int).astype(str), format='%Y%m', errors='coerce')
    
    base_date = pd.to_datetime('2025-12-01')
    df['building_month'] = (base_date - df['temp_time']).dt.days / 30.44
    df['building_month'] = df['building_month'].fillna(df['building_month'].median())
    df['building_month'] = df['building_month'].astype(float)

    # -----------------------------------------------------
    # 3. 地価データの結合 
    # -----------------------------------------------------
    if not df_land.empty:
        df_land['lat'] = df_land.geometry.y
        df_land['lon'] = df_land.geometry.x
        land_clean = df_land[['lat', 'lon', 'price']].dropna()
        
        # データがある場合のみBallTree
        if len(land_clean) > 0:
            land_rad = np.deg2rad(land_clean[['lat', 'lon']])
            input_rad = np.deg2rad(df[['lat', 'lon']])
            
            tree = BallTree(land_rad, metric='haversine')
            dists, indices = tree.query(input_rad, k=1)
            
            df['land_price'] = land_clean['price'].values[indices.flatten()]
            df['dist_to_land_price'] = dists.flatten() * 6371 * 1000
            df['log_land_price'] = np.log1p(df['land_price'])
        else:
             df['log_land_price'] = 0
             df['dist_to_land_price'] = 0
    else:
        df['log_land_price'] = 0
        df['dist_to_land_price'] = 0

    # 4. 地価フィルタ (Trainのみ)
    # 正しい地価データが入っていれば有効化
    if is_train and 'log_land_price' in df.columns:
         df = df[df['log_land_price'] > 5] 

    # -----------------------------------------------------
    # 5. 特徴量リスト
    # -----------------------------------------------------
    cat_cols = [
        'addr1_2', 
        'rosen_name1',
        'eki_name1',
        'building_structure',
        'snapshot_window_direction',
        'madori_kind_all'
    ]
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).fillna('unknown').astype('category')
            
    # 数値特徴量からも station_minutes を削除
    num_features = ['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'station_minutes']
    features = num_features + [c for c in cat_cols if c in df.columns]

    target = None
    if is_train:
        # 単価計算
        unit_price = df['money_room'] / df['unit_area']
        target = np.log1p(unit_price)

    return df, features, target

In [ ]:
# ---------------------------------------------------------
# 1. 前処理 (Train & Test)
# ---------------------------------------------------------
print("前処理を実行中...")

# Trainデータの処理 (学習用なので is_train=True)
# ※ preprocess_data関数はさっき定義した最新版を使ってください
train_processed, features, target = preprocess_data(df, df_land, is_train=True)
# 【Train独自の処理】
# 面積がないデータは「教師として質が悪い」ので、行ごと消す
train_processed = train_processed.dropna(subset=['unit_area'])

# 削除した分、target（目的変数）も合わせる必要があるので、再計算するか
# あるいは preprocess_data の中で dropna してから target を作るのが安全ですが、
# 今の段階なら「埋めるための値（中央値）」をここで計算して保存しておくのが大事です。

# ★Testを埋めるための「正解の値」をキープ！
fill_value = train_processed['unit_area'].median() 


# ---------------------------------------------------------
# 2. Testデータの処理
# ---------------------------------------------------------
# 先に関数を通す
test_processed, _, _ = preprocess_data(test, df_land, is_train=False)

# 【Test独自の処理】
# 提出用なので行は消せない。「Trainの中央値」を使って埋める
test_processed['unit_area'] = test_processed['unit_area'].fillna(fill_value)
print(f"使用する特徴量: {len(features)}個")
print(features)

from sklearn.model_selection import KFold

# ---------------------------------------------------------
# 2. モデル学習 & 予測 (K-Fold アンサンブル)
# ---------------------------------------------------------
print("\nモデル学習 & K-Fold予測中...")

# 5分割する
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# テストデータへの予測値を格納する配列（0で初期化）
test_preds_sum = np.zeros(len(test_processed))
oof_preds = np.zeros(len(train_processed)) # 手元の検証用

# 特徴量リストの整理
use_features = [f for f in features if f in train_processed.columns]
cat_features = [c for c in use_features if train_processed[c].dtype.name == 'category']

# 評価スコア計算用
cv_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(train_processed, target)):
    # データの分割
    X_tr, X_val = train_processed.iloc[train_index][use_features], train_processed.iloc[val_index][use_features]
    y_tr, y_val = target.iloc[train_index], target.iloc[val_index]
    
    # データセット作成
    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=cat_features)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=cat_features)
    
    # 手動パラメータ（Optunaで悪化したなら、一旦これらを使うのが安全）
    params = {
        'objective': 'regression',
        'metric': 'mape',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.05,
        'num_leaves': 64,
        'max_depth': -1,
        'min_child_samples': 20,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
    }
    
    # 学習
    model = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_val],
        num_boost_round=10000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100, verbose=False),
            lgb.log_evaluation(0) # ログを黙らせる
        ]
    )
    
    # 1. 検証データへの予測 (対数を戻してスコア計算)
    val_pred_log = model.predict(X_val)
    oof_preds[val_index] = val_pred_log
    
    # 手元のMAPEスコア確認
    score = mean_absolute_percentage_error(np.expm1(y_val), np.expm1(val_pred_log)) * 100
    cv_scores.append(score)
    print(f"Fold {fold+1} MAPE: {score:.4f}%")
    
    # 2. テストデータへの予測 (加算していく)
    test_preds_sum += model.predict(test_processed[use_features])

# ---------------------------------------------------------
# 結果の集計
# ---------------------------------------------------------
print("="*30)
print(f"平均 CV MAPE: {np.mean(cv_scores):.4f}%")
print("="*30)

# テストデータの予測値を「5で割って平均」にする
avg_pred_log = test_preds_sum / 5

# 対数を戻す (log -> exp)
pred_price_per_m2 = np.expm1(avg_pred_log)

# 単価 × 面積 ＝ 総額
pred_total_price = pred_price_per_m2 * test_processed['unit_area']

# 以下、CSV作成は元のコードと同じでOK

# ---------------------------------------------------------
# 4. 提出用CSVの作成
# ---------------------------------------------------------
print("CSV作成中...")

# id列と予測値をくっつける
submission = pd.DataFrame({
    'id': test['id'],         # テストデータのID
    'prediction': pred_total_price # 予測した総額
})

# CSVに書き出し (header=Falseが必要な場合が多いので注意)
# 今回は「id,値段」の形式ということなので、ヘッダーありかなしか確認してください。
# 多くの場合、SIGNATEなどはヘッダー不要(header=False)か、指定のヘッダー名が必要です。
# ここでは一旦、一般的な「ヘッダーなし」で保存します。
submission.to_csv('submission.csv', index=False, header=False)

print("\n完了！ 'submission.csv' が保存されました。")
print("先頭5行はこんな感じです↓")
print(submission.head())

前処理を実行中...


使用する特徴量: 10個
['unit_area', 'log_land_price', 'dist_to_land_price', 'building_month', 'station_minutes', 'addr1_2', 'rosen_name1', 'eki_name1', 'building_structure', 'madori_kind_all']

モデル学習中...
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001165 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5882
[LightGBM] [Info] Number of data points in the train set: 253379, number of used features: 10
[LightGBM] [Info] Start training from score 12.476830
予測を実行中...
CSV作成中...

完了！ 'submission.csv' が保存されました。
先頭5行はこんな感じです↓
   id    prediction
0   0  1.373621e+07
1   1  3.096080e+07
2   2  1.349060e+07
3   3  2.711284e+07


In [4]:
print(f"もとのデータ数：{len(test)} 行、予測データ数：{len(submission)} 行")

もとのデータ数：112437 行、予測データ数：112437 行


In [5]:
print(submission.isnull().sum())

id            0
prediction    0
dtype: int64


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

def objective(trial):
    # ---------------------------------------------------------
    # 1. チューニングしたいパラメータの範囲を定義
    # ---------------------------------------------------------
    params = {
        'objective': 'regression',
        'metric': 'rmse',      # log(price)のRMSEを最小化すれば、MAPEも良くなる
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        
        # 探索する変数たち
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 32, 256),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }

    # ---------------------------------------------------------
    # 2. 高速な交差検証 (K-Fold)
    # ---------------------------------------------------------
    # 毎回全データでやると重いので、データが多い場合は 3-Fold くらいで探索するのがコツ
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    
    # スコア格納用
    rmses = []
    
    # カテゴリ変数の指定
    # featuresリストに入っているカテゴリ変数を抽出
    cat_features = [c for c in features if train_processed[c].dtype.name == 'category']
    
    # KFoldループ（LightGBMのcv関数を使うともっと速いですが、わかりやすく手動ループにします）
    for train_index, val_index in kf.split(train_processed, target):
        X_train, X_val = train_processed.iloc[train_index][features], train_processed.iloc[val_index][features]
        y_train, y_val = target.iloc[train_index], target.iloc[val_index]
        
        # データセット作成
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_features)
        lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=cat_features)
        
        # Pruning（ダメそうな試行は早めに打ち切る機能）を有効化
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
        
        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_val],
            num_boost_round=10000,  # 多めに設定
            callbacks=[
                lgb.early_stopping(stopping_rounds=50, verbose=False),
                pruning_callback
            ]
        )
        
        # 検証スコア (RMSE) を記録
        preds = model.predict(X_val)
        rmse = np.sqrt(np.mean((preds - y_val)**2))
        rmses.append(rmse)
    
    return np.mean(rmses)

# ---------------------------------------------------------
# 3. 探索実行！ (ここから時間がかかります)
# ---------------------------------------------------------
study = optuna.create_study(direction='minimize') # RMSEを最小化したい
study.optimize(objective, n_trials=50) # 50回試行（時間は調整してください）

print("="*30)
print("Best Valid RMSE:", study.best_value)
print("Best Params:", study.best_params)
print("="*30)

[I 2025-12-20 22:23:57,731] A new study created in memory with name: no-name-98818113-1963-4e7a-9978-7f8e838c1b1f


In [ ]:
## 駅情報のEDA.

print("==駅情報のサンプル表示")
train_processed[['bus_time1', 'walk_distance1']].sample(50)

==駅情報のサンプル表示


,bus_time1,walk_distance1
123380,NaN,1040.0
36050,NaN,1280.0
99127,NaN,1040.0
199249,NaN,400.0
308831,NaN,240.0
20211,NaN,640.0
98858,NaN,320.0
356737,NaN,1200.0
56729,NaN,240.0
295701,NaN,1280.0
